Okay, so the better way to do this would be to actually load this data into postgres and then query it in the database, but for the sake of actually getting the data out we'll do it the slow way and load it into jupyter notebook using pandas.

In [1]:
import pandas as pd
import pickle

# Get pandas and postgres to work together
import psycopg2 as pg
import pandas as pd
import pandas.io.sql as pd_sql

In [3]:
brfss_2014 = pd.read_csv("../data/brfss/brfss2014.csv", encoding = "cp1252", nrows=100)

In [5]:
brfss_2014.head()

,Unnamed: 0,x.state,fmonth,idate,imonth,iday,iyear,dispcode,seqno,x.psu,...,x.fobtfs,x.crcrec,x.aidtst3,x.impeduc,x.impmrtl,x.imphome,rcsbrac1,rcsrace1,rchisla1,rcsbirth
0,1,1,1,1172014,1,17,2014,1100,2014000001,2014000001,...,2.0,1.0,2.0,5,1,1,NaN,NaN,NaN,NaN
1,2,1,1,1072014,1,7,2014,1100,2014000002,2014000002,...,2.0,2.0,2.0,4,1,1,NaN,NaN,NaN,NaN
2,3,1,1,1092014,1,9,2014,1100,2014000003,2014000003,...,2.0,2.0,2.0,6,1,1,NaN,NaN,NaN,NaN
3,4,1,1,1072014,1,7,2014,1100,2014000004,2014000004,...,2.0,1.0,2.0,6,3,1,NaN,NaN,NaN,NaN
4,5,1,1,1162014,1,16,2014,1100,2014000005,2014000005,...,2.0,1.0,2.0,5,1,1,NaN,NaN,NaN,NaN


In [30]:
brfss_2014.rename(columns={'Unnamed: 0': 'record_id'}, inplace=True)

remove_periods_dict = dict([(col, col.replace(".", "")) for col in brfss_2014.columns])

brfss_2014.rename(columns=remove_periods_dict, inplace=True)

In [33]:
brfss_2014_columns = [str(col).lower() + " TEXT" for col in brfss_2014.columns]

brfss_2014_columns_str = ", ".join(brfss_2014_columns)
brfss_2014_create_table_query = "CREATE TABLE brfss_2014 (" + brfss_2014_columns_str + ")"

brfss_2014_load_table = "COPY brfss_2014 FROM '/home/ubuntu/brfss/brfss2014.csv' CSV HEADER ENCODING 'LATIN1'"

In [70]:
brfss_2014_columns

['addepev2 TEXT',
 'alcday5 TEXT',
 'asactlim TEXT',
 'asattack TEXT',
 'asbiadvc TEXT',
 'asbialch TEXT',
 'asbibing TEXT',
 'asbidrnk TEXT',
 'asbirduc TEXT',
 'asdrvist TEXT',
 'aservist TEXT',
 'asinhalr TEXT',
 'asnoslep TEXT',
 'asrchkup TEXT',
 'asthma3 TEXT',
 'asthmage TEXT',
 'asthmed3 TEXT',
 'asthnow TEXT',
 'asymptom TEXT',
 'avedrnk2 TEXT',
 'bldstool TEXT',
 'bldsugar TEXT',
 'blind TEXT',
 'cadult TEXT',
 'carercvd TEXT',
 'casthdx2 TEXT',
 'casthno2 TEXT',
 'cclghous TEXT',
 'cellfon2 TEXT',
 'chccopd1 TEXT',
 'chckidny TEXT',
 'chcocncr TEXT',
 'chcscncr TEXT',
 'checkup1 TEXT',
 'children TEXT',
 'chkhemo3 TEXT',
 'cncrage TEXT',
 'cncrdiff TEXT',
 'cncrtyp1 TEXT',
 'colghous TEXT',
 'cpdemo1 TEXT',
 'csrvclin TEXT',
 'csrvctl1 TEXT',
 'csrvdein TEXT',
 'csrvdoc1 TEXT',
 'csrvinsr TEXT',
 'csrvinst TEXT',
 'csrvpain TEXT',
 'csrvrtrn TEXT',
 'csrvsum TEXT',
 'csrvtrt1 TEXT',
 'cstate TEXT',
 'ctelenum TEXT',
 'ctelnum1 TEXT',
 'cvdcrhd4 TEXT',
 'cvdinfr4 TEXT',
 'cvd

In [34]:
brfss_2014_create_table_query

'CREATE TABLE brfss_2014 (record_id TEXT, xstate TEXT, fmonth TEXT, idate TEXT, imonth TEXT, iday TEXT, iyear TEXT, dispcode TEXT, seqno TEXT, xpsu TEXT, ctelenum TEXT, pvtresd1 TEXT, colghous TEXT, stateres TEXT, ladult TEXT, numadult TEXT, nummen TEXT, numwomen TEXT, genhlth TEXT, physhlth TEXT, menthlth TEXT, poorhlth TEXT, hlthpln1 TEXT, persdoc2 TEXT, medcost TEXT, checkup1 TEXT, exerany2 TEXT, sleptim1 TEXT, cvdinfr4 TEXT, cvdcrhd4 TEXT, cvdstrk3 TEXT, asthma3 TEXT, asthnow TEXT, chcscncr TEXT, chcocncr TEXT, chccopd1 TEXT, havarth3 TEXT, addepev2 TEXT, chckidny TEXT, diabete3 TEXT, diabage2 TEXT, lastden3 TEXT, rmvteth3 TEXT, veteran3 TEXT, marital TEXT, children TEXT, educa TEXT, employ1 TEXT, income2 TEXT, weight2 TEXT, height3 TEXT, numhhol2 TEXT, numphon2 TEXT, cpdemo1 TEXT, internet TEXT, renthom1 TEXT, sex TEXT, pregnant TEXT, qlactlm2 TEXT, useequip TEXT, blind TEXT, decide TEXT, diffwalk TEXT, diffdres TEXT, diffalon TEXT, smoke100 TEXT, smokday2 TEXT, stopsmk2 TEXT, las

Now that we have this table for brfss_2014 loaded, let's go ahead and query the database for the information that is interesting.

In [2]:
# Postgres info to connect
connection_args = {
    'host': input(), # my public ip address
    'user': 'ubuntu',    # username
    'dbname': 'brfss',   # DB that we are connecting to
    'port': 5432         # port we opened on AWS
}

connection = pg.connect(**connection_args)

54.67.118.82


In [3]:
diabetes_query = "SELECT xstate, diabete3 FROM brfss_2014"

In [4]:
diabetes_df = pd_sql.read_sql(diabetes_query, connection)

In [5]:
diabetes_df[diabetes_df.xstate == '1']['diabete3'].value_counts()

3    6892
1    1528
4     157
2      62
7       9
9       4
Name: diabete3, dtype: int64

In [59]:
diabetes_df[diabetes_df.xstate == '1']['diabete3'].value_counts(normalize=True)

3    0.796579
1    0.176607
4    0.018146
2    0.007166
7    0.001040
9    0.000462
Name: diabete3, dtype: float64

In [6]:
diabetes_weights_query = "SELECT xstate, diabete3, xllcpwt2, xllcpwt FROM brfss_2014"

In [7]:
diabetes_weights_df = pd_sql.read_sql(diabetes_weights_query, connection)

In [8]:
diabetes_weights_df.head()

,xstate,diabete3,xllcpwt2,xllcpwt
0,1,3,124.3773858077,266.861265396154
1,1,3,121.624464566304,77.5983856870883
2,1,3,175.953264468735,108.943346379677
3,1,3,263.929896703102,259.031555509853
4,1,3,263.929896703102,334.597982387287


In [66]:
(1528)/8652

0.17660656495607951

In [58]:
diabetes_df[diabetes_df.xstate == '1'].shape

(8652, 2)

Now we'll use psycopg2 and sqlalchemy to upload the brfss_2014 dataframe to psql, which will create the schema. Then we'll copy the csv to the table that got created.

In [14]:
# Postgres info to connect
connection_args = {
    'host': input(), # my public ip address
    'user': 'ubuntu',    # username
    'dbname': 'texas_discharge',   # DB that we are connecting to
    'port': 5432         # port we opened on AWS
}

connection = pg.connect(**connection_args)

In [21]:
connection = connect(**params, dbname='ubuntu')

In [16]:
# Connect and create database, disconnect, and reconnect to the right database
connection = connect(**params, dbname='ubuntu')
connection.set_isolation_level(ISOLATION_LEVEL_AUTOCOMMIT)
connection.cursor().execute('CREATE DATABASE brfss;')
connection.close()

In [22]:
from sqlalchemy import create_engine
connection_string = f'postgres://ubuntu:{params["host"]}@{params["host"]}:{params["port"]}/brfss'
engine = create_engine(connection_string)
brfss_2014.to_sql('brfss_2014', engine, index=False)